In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pylab as pylab
import pandas as pd
from operator import itemgetter

from warnings import filterwarnings

pylab.rcParams['figure.figsize'] = 5, 5
plt.rc("font", size=10)


plt.rc('xtick', color='k', labelsize='medium', direction='in')
plt.rc('xtick.major', size=8, pad=12)
plt.rc('xtick.minor', size=8, pad=12)

plt.rc('ytick', color='k', labelsize='medium', direction='in')
plt.rc('ytick.major', size=8, pad=12)
plt.rc('ytick.minor', size=8, pad=12)

In this chapter we will use a simple regression case to introduce some **core concepts of Supervised Learning and  Machine Learning**. 



![image showing the difference between classification (left) and regression (right). On the left, set of red and blue points are separated by a black line which represent our model. In contrast, the model's line follow along the patterns of the points cloud](../images/class_reg.png)

Supervised Learning considers tasks where we want to predict a predefined target. If the target is categorical we talk about a **classification task**. If the target is continuous we talk about **regression task**.


Regression and classification use different models and optimize different metric,
but in Machine Learning they also share many common practices and concerns.


The machine learning paradigm emphasizes the importance of building a model that will be good 
**not only on the data it has been trained on, but also on new data**.
In other words, we want a model that can be **generalized to new data**.







<br>

To introduce this as well as some of the core concepts of Machine Learning, we will illustrate with a relatively *simple* case of a linear modelling.

---

# Table Of Content: <a id="toc"></a>


* [**motivating example**](#motivation)
* [approach 1: a simple linear regression](#linear-1)
* [approach 2: adding regularization and validation set](#linear-2)
* [approach 3 : k-fold cross-validation](#linear-3)
* [approach 4 : a "classical" ML pipeline](#linear-4)
    


# motivating example <a id="motivation"></a>

[Acharjee et al.2016](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1043-4) propose several -omic dataset which they used to predict and gain knowledge on various phenotypic traits in potatos.

Here, we will concentrate on their transcriptomics dataset and the phenotypic trait of the potato coloration.

We have pre-selected and normalized the 200 most promising genes (out of ~15 000).

In [ ]:
file_metadata = "../data/potato_data.phenotypic.csv"
file_data = "../data/potato_data.transcriptomic.top200norm.csv"

df = pd.read_csv( file_metadata , index_col=0 )
dfTT = pd.read_csv( file_data , index_col=0)

df.shape

For the sake of our story, we will imagine that out of the 86 potatos in the data, we have only 73 at the time of our experiment.

We put aside the rest for later.

In [ ]:
i1 = df.index[:73]
i2 = df.index[73:]

In [ ]:
X = dfTT.loc[i1 , :]
X.head()

In [ ]:
y = df.loc[i1 , "Flesh Colour"]
y.describe()

In [ ]:
sns.histplot(y)

[Back to ToC](#toc)

## approach 1: a simple linear regression <a class="anchor" id="linear-1"></a>


We will be modelling the potato flesh color using a simple [linear model](https://en.wikipedia.org/wiki/Linear_model),
where we would write the equation of the flesh color something like:


$$flesh color = \beta_1 * E1 + \beta_2 * E2 + ... + \beta_n * En $$

Where $Ex$ is the expression level of gene $x$, and $\beta_x$ is the associated coefficient (which describes how much the expression of gene $x$ contributes to the flesh color prediction).



> It is very likely that you have already seen and manipulated linear models previously, and we are not going to develop much further here. But in case this is the first time you meet them, or you just want a refresher, here is a [jupyter notebook on linear model](https://github.com/sib-swiss/statistics-and-machine-learning-training/blob/main/01_regression_Least_Square.ipynb), from one of our other courses ;-) 


---


Let's fit a simple linear model with our gene expression values, and see what happens


For this, we will use the `sklearn` syntax:

In [ ]:
## we import elements from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score , mean_squared_error


# create the regression object
lin_reg = LinearRegression()

# fit it with our data
lin_reg.fit(X,y)

# predict
y_pred = lin_reg.predict( X )

# evaluate the prediction
print(f"R-squared score: { r2_score( y , y_pred ) :.2f}")
print(f"mean squared error: { mean_squared_error( y , y_pred ) :.2f}")


**Wow!!** this is a perfect fit.

But if you know anything about biology, or data analysis, then you likely suspect something wrong is happening.


Indeed, at the moment, our claim is that our model can predict flesh color perfectly (RMSE=0.0) from the normalized expression of these 200 genes.

But, say we now have some colleagues who come to us with some new potato data:


In [ ]:
## now we use the leftover data points:
Xnew = dfTT.loc[i2 , :]
ynew = df.loc[i2 , "Flesh Colour"]

## apply the model on the new data
ynew_pred = lin_reg.predict( Xnew )

# evaluate the prediction
print(f"new data R-squared score: { r2_score( ynew , ynew_pred ) :.2f}")
print(f"new data mean squared error: { mean_squared_error( ynew , ynew_pred ) :.2f}")

In [ ]:
plt.scatter( y , y_pred , label = 'training data' )
plt.scatter( ynew , ynew_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

As expected, the performance on the new data is not as good as with the data we used to train the model.

We have **overfitted** the data.

---

Here we could still use the model that we have created, 
but we would agree that reporting the perfect performance we had with our training data would be misleading.

To honestly report the performance of our model, we measure it on a **set of data that has not been used at all to train it: the *test set*.**



To that end, we typically begin by dividing our data into :

 * **train** set : find the best model
 * **test** set  : give an honest evaluation of how the model perform on completely new data.

![train_test](../images/train_test.png)

In [ ]:
## let's rename the python objects to reflect this concept:

X_test = Xnew
y_test = ynew

[Back to ToC](#toc)

## approach 2: adding regularization and validation set <a class="anchor" id="linear-2"></a>

In the case of a Least Square fit, the function you are minimizing looks like:

$\sum_i (y_i-y\_pred_i))^2$

, so the sum of squared difference between the observation and the predictions of your model.


And remember that our model is written in the following way

$$y\_pred_i = \beta_1 * X1_{i} + \beta_2 * X2_{i} + ... + \beta_n * Xn_{i} $$

---

**Regularization** is a way to reduce overfitting, and in the case of the linear model
we do so by adding to this function a **penalization term which depends on coefficient weights** (*ie*, the $\beta_x$).

In brief, the stronger the coefficient, the higher the penalization. So only coefficients which bring more fit than penalization will be kept.


> Note : we report here the formulas used in `scikit-learn` functions. Other libraries may have a different parameterization, but the concepts stay the same


$\frac{1}{2n}\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}|\beta_{j}|$ , **l1 regularization** (Lasso) $\alpha$ being the weight that you put on that regularization 


$\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}\beta_{j}^{2}$ , **l2 regularization** (Ridge) 


$\frac{1}{2n}\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}(\rho|\beta_{j}|+\frac{(1-\rho)}{2}\beta_{j}^{2})$ , **elasticnet**


For a deeper understanding of those notions, you may look at :

 * [datacamp tutorial](https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net)

 * [towardsdatascience tutorial](https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a)



> NB: Regularization generalize to maximum likelihood contexts as well)




You can see there is a **parameter $\alpha$: the regularization strength**

A small $\alpha$ will give you almost the same model as before.

A large $\alpha$ will give you a model where only a few coefficient are allowed to have a significant impact on the prediction.


Let's try that on our data:

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor     
# we import the Stochastic Gradient Descent Regressor
# which implements the algorithm to find the best coefficient 
# when there is a regularization penalty

logalphas = []

coef_dict = {'name' : [],
             'coefficient' : [],
             'log-alpha' : []}
r2 = []

## we try different values of alpha. 
for alpha in np.logspace(-2,2,50):

    reg = SGDRegressor( penalty='l1' , alpha = alpha ) # create the model 
    reg.fit( X , y )                                   # fit the model 
    
    logalphas.append(np.log10(alpha))
    r2.append( r2_score( y , reg.predict(X) ) )       # compute R2 score
    
    
    ## I keep the value of the coefficients to show what the regularization does
    coef_dict['name'] += list( X.columns )
    coef_dict['coefficient'] += list( reg.coef_ )
    coef_dict['log-alpha'] += [np.log10(alpha)]* len(X.columns )

coef_df = pd.DataFrame(coef_dict)

In [ ]:

fig,ax = plt.subplots(1,2,figsize = (14,7))

ax[0].plot(logalphas , r2)
ax[0].set_xlabel("log10( alpha )")
ax[0].set_ylabel("R2")

sns.lineplot( x = 'log-alpha' , y='coefficient' , hue = 'name' , data= coef_df , ax = ax[1] ,legend = False)

fig.suptitle("regression of potato data with an L1 regularization.")

On the left subplot you see how the R-squared score changes with $log10(\alpha)$.


 * **High $R^2$ at low $\alpha$**: at lower values of $log10(\alpha)$ (left side of the plot), the R-squared score is high, indicating that the model fits the data well. This is because the regularization is weak, allowing more features to contribute to the model.
 * **Sharp Drop in $R^2$**: as $log10(\alpha)$ increases, the R-squared score drops sharply. This indicates that stronger regularization (higher alpha values) leads to a simpler model with fewer features, which may not capture the data’s variance as well.
 * **$R^2 = 0$ at high $\alpha$**: at higher values of $log10(\alpha)$ (right side of the plot), the R-squared score is 0, indicating that regularization is so strong that no coefficient are contributing to the predictions (*ie*, their values are all at 0).


The right subplot confirms this interpretation as it shows the coefficients values for each of level of $\alpha$

 * Each line represents the path of a coefficient for a specific feature (here a gene) as $\alpha$ varies.
 * As log-alpha increases, many **coefficients shrink towards zero**. This is the effect of L1 regularization, which penalizes the absolute size of the coefficients, leading to sparse models where many coefficients become exactly zero.
 * Features with coefficients that shrink to zero first are less important for the model. Features whose **coefficients remain non-zero for higher values of log-alpha are more important for the model**.






**Micro-exercise:** adapt the code above to generate this plot with an l2 penalty. How do you interpret the difference?


In [ ]:
# %load solutions/solution_03_mini1.py

This is great, but how do we choose which level of regularization we want ?

It is a general rule that **as you decrease $\alpha$, the $R^2$ on the data used for the fit increase**, i.e. you risk overfitting.

Consequently, we cannot choose the value of $\alpha$ parameter from the data used to fit alone; we call such a parameter an **hyper-parameter**.

**Question:** what are other hyper-parameters at this point?

---

<br>

In order to find the optimal value of an hyper-parameter, we can separate our training data into:
 * a **train set** : used to fit the model
 * a **validation set** : used to evaluate how our model perform on new data 


In [ ]:
X.shape

In [ ]:
%%time
## adding a validation set

# we will use 60 points to train the model
# and we will use the rest to evaluate the model 
I = list( range(X.shape[0]))
np.random.shuffle( I ) 

I_train = I[:60]
I_valid = I[60:]

X_train = X.iloc[ I_train , : ] 
y_train = y.iloc[ I_train ]

# we will use the rest to evaluate the model
X_valid = X.iloc[ I_valid , : ] 
y_valid = y.iloc[ I_valid ]


logalphas = []

r2_train = []
r2_valid = []

for alpha in np.logspace(-3,2,200):

    reg = SGDRegressor( penalty='l1' , alpha = alpha  )
    reg.fit( X_train , y_train )
    
    logalphas.append(np.log10(alpha))
    r2_train.append( r2_score( y_train , reg.predict(X_train) ) )
    r2_valid.append( r2_score( y_valid , reg.predict(X_valid) ) )
    
## plotting and reporting 
bestI = np.argmax(r2_valid)
bestLogAlpha = logalphas[bestI]
bestR2_valid = r2_valid[bestI]

fig,ax = plt.subplots(figsize = (10,5))
fig.suptitle("best alpha : {:.2f} - validation R2 : {:.2f}".format(10**bestLogAlpha , bestR2_valid))
ax.plot( logalphas, r2_train , label='train set' )
ax.plot( logalphas, r2_valid , label='validation set' )
ax.scatter( [bestLogAlpha] , [bestR2_valid]  , c='red')
ax.set_xlabel("log10( alpha )")
ax.set_ylabel("R2")
ax.set_ylim(-0.1,1.1)
ax.legend()


So now, with the help of a validation set, we can clearly see the phases :
 * **underfitting** : for high $\alpha$, the performance is low for both the train and the validation set
 * **overfitting** : for low $\alpha$, the performance is high for the train set, and low for the validation set
 
We want the equilibrium point between the two where performance is ideal for the validation set.

**Problem :** if you run the code above several time, you will see that the optimal point varies due to the random assignation to train or validation set. 

There exists a myriad of possible strategies to deal with that problem, such as repeating the above many times and taking the average of the results for instance.

> Note also that this problem gets less important as the validation set size increases.

<br>

---

<br>

Anyhow, on top of our earlier regression model, we have added :

 * an **hyper-parameter** : $\alpha$, the strength of the regularization term
 * a **validation strategy** for our model in order to avoid overfitting

<br>

That's it, we are now in the world of Machine Learning.

But before we go any further, let's see how this modified model performs on the test data:

In [ ]:

reg = SGDRegressor( penalty='l1' , alpha = 10**bestLogAlpha  )
reg.fit( X , y )

y_pred = reg.predict( X )
print(f"train data R-squared score: { r2_score( y , y_pred ) :.2f}")
print(f"train data mean squared error: { mean_squared_error(  y , y_pred ) :.2f}")


y_test_pred = reg.predict( X_test )

print(f" test data R-squared score: { r2_score( y_test , y_test_pred ) :.2f}")
print(f" test data mean squared error: { mean_squared_error(  y_test , y_test_pred ) :.2f}")


plt.scatter( y , y_pred , label = 'training data' )
plt.scatter( y_test , y_test_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

Two things to observe:
 * we still see better performance on the train data than on the test data (generally always the case)
 * the performance on the test set has improved: our model is less overfit and more generalizable

[Back to ToC](#toc)

##  approach 3 : k-fold cross-validation <a class="anchor" id="linear-3"></a>

In the previous approach, we have split our training data into a train set and a validation set.

This approach works well if you have enough data for your validation set to be representative.

Often, we unfortunately do not have enough data for this.

Indeed, we have seen that if we run the code above several time, we see that the optimal point varies due to the random assignation to train or validation set. 


**K-fold cross validation** is one of the most common strategy to try to mitigate this randomness with a limited amount of data.

![k-fold validation](../images/kfold.png)

In k-fold cross-validation

1. the data is split in $k$ subparts, called fold.
2. $k$ models are trained for each hyper-parameter values combination: each time a different fold for validation (and the remaining $k-1$ folds for training).
3. the average performance is computed across all fold : this is the **cross-validated performance**.

--- 

We are going to do a simple k-fold manually once, to explore a bit how it works, but in practice you will discover that it is mostly automatized with some of scikit-learn's recipes and objects.

In [ ]:
## Kfold
from sklearn.model_selection import KFold



kf = KFold(n_splits=5 , shuffle=True , random_state=734)
for i, (train_index, valid_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={valid_index}")


In [ ]:
%%time
logalphas = np.linspace(-2,1,200)

kf = KFold(n_splits=5 , shuffle=True , random_state=6581) ## try changing the random state

fold_r2s = [ [] for i in range(kf.n_splits) ] ## for each fold
cross_validated_r2 = [] # average across folds

for j,alpha in enumerate( 10**logalphas ) :                       # hyper-parameter loop

    cross_validated_r2.append(0)
    
    for i, (train_index, valid_index) in enumerate(kf.split(X)):  # k-fold loop

        ## split train and validation sets
        X_train = X.iloc[ train_index , : ]
        X_valid = X.iloc[ valid_index , : ]

        y_train = y.iloc[ train_index ]
        y_valid = y.iloc[ valid_index ]

        ## fit model for that fold
        reg = SGDRegressor( penalty='l1' , alpha = alpha  )
        reg.fit( X_train , y_train )

        ## evaluate for that fold
        fold_score = r2_score( y_valid , reg.predict(X_valid) )
        
        ## keeping in the curve specific to this fold
        fold_r2s[i].append( fold_score )
        
        ## keeping a tally of the average across folds
        cross_validated_r2[-1] += fold_score/kf.n_splits

        
        

bestI = np.argmax(cross_validated_r2)
bestLogAlpha = logalphas[bestI] 
bestR2_valid = cross_validated_r2[bestI]


## plotting
fig,ax = plt.subplots(figsize = (10,5))

ax.plot( logalphas, cross_validated_r2 , label='cross-validated r2' )
ax.scatter( [bestLogAlpha] , [bestR2_valid]  , c='red')

for i,scores in enumerate(fold_r2s):
    ax.plot( logalphas , scores , label = f'fold {i}' , linestyle='dotted' )

ax.set_xlabel("log10( alpha )")
ax.set_ylabel("R2")
ax.set_ylim(-0.1,1.1)
ax.legend()

fig.suptitle("best alpha : {:.2f} - cross-validated R2 : {:.2f}".format(10**bestLogAlpha , bestR2_valid))



**micro-exercise**: re-fit a model with the alpha we found and check the performance with the *test* data

In [ ]:
# %load solutions/solution_03_mini2.py

There, you can realize that now, for each possible value of our hyper-parameter we fit and evaluate not 1, but $k$ models, here 4.

So, for 200 values of $\alpha$, that means 200x5 = 1000 models to fit and evaluate.

Now, consider that we have other hyper-parameters, such as the type of regularization (L1 or L2),
or how we perform scaling, or whether we consider interactions, and now you understand why Machine Learning can quickly become  computationally intensive. 

---


[Back to ToC](#toc)

## approach 4 : a "classical" ML pipeline <a class="anchor" id="linear-4"></a>

We will start back from scratch to recapitulate what we've seen and use scikit-learn's helpful recipes to solve the potato problem.


In [ ]:
## full dataset
X = dfTT
y = df[ "Flesh Colour"]


We start by splitting our data in a train and a test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.2 )

print('train set size:',len(y_train))
print(' test set size:',len(y_test))

--- 
Here comes the question : **"What should be my test-set size?"** 

There is not a single clear-cut answer to this and it depends a lots on the nature of your data and the of the problem you want to solve.


On one hand, if you have **too few training samples**: your model will have a hard time finding the patterns in the data and will likily have poor permance.

On one hand, if you have **too few testing samples**: the evaluation of your model will be very imprecise.

The typical thing you will see around is 80% training / 20% testing, or 75%/25% **BUT**

here are some of things you want to keep in mind :

 * consider the **number of samples** you put in the testing set rather than the proportion
 * you want the test set to be **representative**: how much samples do you need for that?
 * what sort of precision would you like to have on the evaluation of your model?
     You can use [proportion confidence intervals](https://www.statskingdom.com/proportion-confidence-interval-calculator.html) to help you
 * **how hard is your problem?** Some problems need a lot of data, and some needs a lot.
 
You can read more on this subject in this [blog post](https://www.r-bloggers.com/2021/01/what-is-a-good-test-set-size/). 

---

Now we train a model while optimizing some hyper-parameters.

On top of what we've done before, I add a scaling phase, and test l1 or l2 penalties.

Scikit-learn's `GridSearchCV` is useful to explore these more "complex" hyper-parameter spaces.


In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



## our pipeline will have 2 consecutive steps
##   * standard scaling : set mean of each feature at 0 and (optionally) standard dev at 1 
##   * linear regression with some regularization
pip_reg = Pipeline([('scaler',StandardScaler()),
                    ('model',SGDRegressor())])



# define the hyperparameters you want to test
# with the range over which you want it to be tested.
# 
# They are given in a dictionary with the structure:
#      pipelineStep__parameter : [set of values to explore]
#                  ^^
#                  note the double underscore _
grid_values = {'scaler__with_std' : [ True , False ],
               'model__penalty':[ 'l1' , 'l2' ],
               'model__alpha':np.logspace(-2,2,200)}


# Feed the pipeline and set of values to the GridSearchCV with the 
# score over which the decision should be taken (here, R^2).
# and the cross-validation scheme, here the number of fold in a stratified k-fold strategy
grid_reg = GridSearchCV(pip_reg, 
                        param_grid = grid_values, 
                        scoring='r2', 
                        cv = 5,
                        n_jobs=-1)

# When the actual fit happens
#  the gridSearchCV object will go through each hyperparameter value combination
#  and fit + evaluate each fold, and averages the score across each fold.
#
#  It then finds the combination that gave the best score and
#  use it to re-train a model with the whole train data
grid_reg.fit(X_train, y_train)

# get the best cross-validated score 
print(f'Grid best score ({grid_reg.scoring}): {grid_reg.best_score_:.3f}')

# print the best parameters
print('Grid best parameter :')
for k,v in grid_reg.best_params_.items():
    print(' {:>20} : {}'.format(k,v))


In [ ]:
## gridSearch CV fits a new estimator with the best hyperparameter values
reg = grid_reg.best_estimator_

y_pred = reg.predict( X_train )
print(f"train data R-squared score: { r2_score( y_train , y_pred ) :.2f}")
print(f"train data mean squared error: { mean_squared_error(  y_train , y_pred ) :.2f}")


y_test_pred = reg.predict( X_test )

print(f" test data R-squared score: { r2_score( y_test , y_test_pred ) :.2f}")
print(f" test data mean squared error: { mean_squared_error(  y_test , y_test_pred ) :.2f}")



plt.scatter( y_train , y_pred , label = 'training data' )
plt.scatter( y_test , y_test_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

One can also access the best model and it's coefficients:

In [ ]:
grid_reg.best_estimator_

In [ ]:
## coefficient of the linear model
grid_reg.best_estimator_['model'].coef_

One can also access the CV results for each hyper-parameter values:

In [ ]:
df_cv = pd.DataFrame(grid_reg.cv_results_)
df_cv

In [ ]:
p = sns.lineplot( df_cv , 
                 x = 'param_model__alpha' , 
                 y = 'mean_test_score' , 
                 hue = df_cv.param_model__penalty + ' - std scaling:'+ df_cv.param_scaler__with_std.astype(str) , 
                )
                
p.set_xscale("log")